In [1]:
!pip install flask

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import logging
import os
import typing
import flask

from flask import Flask
from flask import request

print("Starting Battlesnake Server...")

def run_server(handlers: typing.Dict):
    app = Flask(__name__)

    @app.get("/")
    def on_info():
        print("info")
        return handlers["info"]()

    @app.post("/start")
    def on_start():
        print("go!")
        game_state = request.get_json()
        handlers["start"](game_state)
        return "ok"

    @app.post("/move")
    def on_move():
        game_state = request.get_json()
        return handlers["move"](game_state)

    @app.post("/end")
    def on_end():
# This isn't being called by the CLI atm
#        print(request.get_json())
#        end_game = request.get_json()
#        print(end_game["death"])
        handlers["end"](end_game)
        return "ok"

    @app.after_request
    def identify_server(response):
        response.headers.set(
            "server", "battlesnake/github/starter-snake-python"
        )
        return response


    host = "::"
    port = int(os.environ.get("PORT", "8000"))

    logging.getLogger("werkzeug").setLevel(logging.ERROR)

    print(f"\nRunning Battlesnake at http://{host}:{port}")
    app.run(host=host, port=port)

In [ ]:
# Welcome to
# __________         __    __  .__                               __
# \______   \_____ _/  |__/  |_|  |   ____   ______ ____ _____  |  | __ ____
#  |    |  _/\__  \\   __\   __\  | _/ __ \ /  ___//    \\__  \ |  |/ // __ \
#  |    |   \ / __ \|  |  |  | |  |_\  ___/ \___ \|   |  \/ __ \|    <\  ___/
#  |________/(______/__|  |__| |____/\_____>______>___|__(______/__|__\\_____>
#
# This file can be a nice home for your Battlesnake logic and helper functions.
#
# To get you started we've included code to prevent your Battlesnake from moving backwards.
# For more info see docs.battlesnake.com

import random
import typing

# info is called when you create your Battlesnake on play.battlesnake.com
# and controls your Battlesnake's appearance
# TIP: If you open your Battlesnake URL in a browser you should see this data
def info() -> typing.Dict:
    print("INFO")
#MMMM   CUSTOMIZATIONS
    return {
        "apiversion": "1",
        "author": "MaybeMadelyn",  # TODO: Your Battlesnake Username
        "color": "#d5dbdb",  # TODO: Choose color
        "head": "default",  # TODO: Choose head
        "tail": "default",  # TODO: Choose tail
    }


# start is called when your Battlesnake begins a game
def start(game_state: typing.Dict):
    global start_snake_count
    start_snake_count = len(game_state['board']['snakes'])
    print("GAME START with ", start_snake_count," snakes")


# end is called when your Battlesnake finishes a game
# this is not happening right now :/
def end(end_game: typing.Dict):
#    ending = end_game['death']
    print("GAME OVER. You died by: ",ending)    

    
def search(targets, my_head, move_points, points):
    if targets:
        closest = targets[0]
        distance = 100
        
        for target in targets:
            x_distance = abs(my_head["x"] - target["x"])
            y_distance = abs(my_head["y"] - target["y"])
            
            if x_distance + y_distance < distance:
                closest = target
                distance = x_distance + y_distance
                
        if closest["x"] < my_head["x"]:
            move_points["left"] += points
        if closest["x"] > my_head["x"]:
            move_points["right"] += points
        if closest["y"] < my_head["y"]:
            move_points["down"] += points
        if closest["y"] > my_head["y"]:
            move_points["up"] += points
            
            
def search_head(targets, my_head, move_points, points):
    if targets:
        closest = targets[0]
        distance = 100
        
        for target in targets:
            x_distance = abs(my_head["x"] - target["x"])
            y_distance = abs(my_head["y"] - target["y"])
            
            if x_distance + y_distance < distance:
                closest = target
                distance = x_distance + y_distance
                
        if closest["x"] < my_head["x"]:
            move_points["left"] += points
        if closest["x"] > my_head["x"]:
            move_points["right"] += points
        if closest["y"] < my_head["y"]:
            move_points["down"] += points
        if closest["y"] > my_head["y"]:
            move_points["up"] += points
            

    search(game_state["board"]["food"], my_head, move_points, 20)
    search(game_state["board"]["snakes"], my_head, move_points, 20)         
#mmmm FLOODFILL FUNCTION
def floodfill(snakes,start):
    empty_spaces = 0
    visited = []
    to_visit = [start]                         
    unsafe = []
    for snake in snakes:
        for body_part in snake["body"]:
            unsafe.append(body_part) #mmmm APPEND means adding something to the list
    in_bounds = True
    while to_visit:   
        in_bounds = not(to_visit[0]["y"] < 0 or to_visit[0]["y"] > 10 or to_visit[0]["x"] < 0 or to_visit[0]["x"] > 10)
        if to_visit[0] not in unsafe and in_bounds and to_visit[0] not in visited:
            empty_spaces += 1
            to_visit.append({"x": to_visit[0]["x"] +1, "y": to_visit[0]["y"]})
            to_visit.append({"x": to_visit[0]["x"] -1, "y": to_visit[0]["y"]})
            to_visit.append({"x": to_visit[0]["x"], "y": to_visit[0]["y"] +1})
            to_visit.append({"x": to_visit[0]["x"], "y": to_visit[0]["y"] -1})
            
        visited.append(to_visit[0])
        to_visit.pop(0)
        
    return empty_spaces

            
# move is called on every turn and returns your next move
# Valid moves are "up", "down", "left", or "right"
# See https://docs.battlesnake.com/api/example-move for available data
def move(game_state: typing.Dict) -> typing.Dict:

    move_points = {"up": 10, "down": 10, "left": 10, "right": 10}
    moves = ["up", "down", "left", "right"]

    # We've included code to prevent your Battlesnake from moving backwards
    my_head = game_state["you"]["body"][0]  # Coordinates of your head
    my_neck = game_state["you"]["body"][1]  # Coordinates of your "neck"
    my_body = game_state['you']['body']

    if my_neck["x"] < my_head["x"]:  # Neck is left of head, don't move left
        move_points["left"] = 0

    elif my_neck["x"] > my_head["x"]:  # Neck is right of head, don't move right
        move_points["right"] = 0

    elif my_neck["y"] < my_head["y"]:  # Neck is below head, don't move down
        move_points["down"] = 0

    elif my_neck["y"] > my_head["y"]:  # Neck is above head, don't move up
        move_points["up"] = 0

   #MMMM MAKE SURE YOU DONT HIT THE WALL

    game_height = game_state["board"]["height"]
    game_width = game_state["board"]["width"]
    
    
    if my_head["y"] == game_height - 1:
        move_points["up"] -= 1000
    
    if my_head["y"] == 0:
        move_points["down"] -= 1000
        
    if my_head["x"] == game_width - 1:
        move_points["right"] -= 1000
        print("DONT GO RIGHT!")
        print(f"Right points: {move_points['right']}")

    if my_head["x"] == 0:
        move_points["left"] -= 1000
        
        
    #MMMM DONT RUN INTO YOURSELF
    
    for body_part in my_body:
        if body_part["y"] == my_head["y"]+1 and body_part["x"] == my_head["x"]:
            move_points["up"] -= 1000
            
        if body_part["y"] == my_head["y"]-1 and body_part["x"] == my_head["x"]:
            move_points["down"] -= 1000
            
        if body_part["x"] == my_head["x"]+1 and body_part["y"] == my_head["y"]:
            move_points["right"] -= 1000
            
        if body_part["x"] == my_head["x"]-1 and body_part["y"] == my_head["y"]:
            move_points["left"] -= 1000
            
    #MMMM DONT RUN INTO OTHER SNAKES
    
    all_snakes = game_state["board"]["snakes"]
    for snakes in all_snakes:
        for body_part in snakes["body"]:
            if body_part["y"] == my_head["y"]+1 and body_part["x"] == my_head["x"]:
                move_points["up"] -= 1000

            if body_part["y"] == my_head["y"]-1 and body_part["x"] == my_head["x"]:
                move_points["down"] -= 1000

            if body_part["x"] == my_head["x"]+1 and body_part["y"] == my_head["y"]:
                move_points["right"] -= 1000

            if body_part["x"] == my_head["x"]-1 and body_part["y"] == my_head["y"]:
                move_points["left"] -= 1000
                
    move_points["right"] += floodfill(all_snakes, {"x": my_head["x"] +1, "y": my_head["y"]})
    move_points["left"] += floodfill(all_snakes, {"x": my_head["x"] -1, "y": my_head["y"]})
    move_points["up"] += floodfill(all_snakes, {"x": my_head["x"] , "y": my_head["y"] +1})
    move_points["down"] += floodfill(all_snakes, {"x": my_head["x"] , "y": my_head["y"] -1})

    
    #mmmm EAT SOME FOOD         
    #bfs
                
    # Compare all of the moves to create a list of all moves with the highest score
    print(f"Right points: {move_points['right']}")
    best_moves = []
    for move, points in move_points.items():
        if len(best_moves) == 0:
            best_moves.append(move)
        else:
            for good_move in best_moves:
                if points > move_points[good_move]:
                    best_moves.clear()
                    best_moves.append(move)
                elif points == move_points[good_move]:
                    best_moves.append(move)
                    break
                    
    # Choose a random move from the best choices
    print(f"Right points: {move_points['right']}")
    next_move = random.choice(best_moves)
    # TODO: Step 4 - Move towards food instead of random, to regain health and survive longer
    # food = game_state['board']['food']

    print(f"MOVE {game_state['turn']}: {next_move}")
    print(f"MOVE {game_state['turn']} POINTS: UP - {move_points['up']}, DOWN - {move_points['down']}, LEFT - {move_points['left']}, RIGHT - {move_points['right']}")
    return {"move": next_move}

run_server({"info": info, "start": start, "move": move, "end": end})

